In [ ]:
import os
import glob
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model, datasets
from warnings import simplefilter
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string


simplefilter(action='ignore', category=FutureWarning)

train_positive_path = '../Sentiment_Analysis/aclImdb/train/pos/'
train_negative_path = '../Sentiment_Analysis/aclImdb/train/neg/'
test_positive_path = '../Sentiment_Analysis/aclImdb/test/pos/'
test_negative_path = '../Sentiment_Analysis/aclImdb/test/neg/'

def read_files_in_path(path):
    positive_training_data=[]
    for filepath in glob.glob(os.path.join(path, '*.txt')):
        with open(filepath,encoding='utf8') as f:
            content = f.read()
            positive_training_data.append(content)
    return positive_training_data

#Read training data
positive_training_data = read_files_in_path(train_positive_path)
print(len(positive_training_data))
negative_training_data = read_files_in_path(train_negative_path)
print(len(negative_training_data))
whole_training_data = positive_training_data+negative_training_data
print(len(whole_training_data))

#Reading Test Data
positive_test_data = read_files_in_path(test_positive_path)
print(len(positive_test_data))
negative_test_data = read_files_in_path(test_negative_path)
print(len(negative_test_data))
whole_test_data = positive_test_data+negative_test_data
print(len(whole_test_data))

In [53]:
#Remove HTML tags from dataset
def cleanhtml(whole_training_data):
    no_HTML_whole_training_data = []
    cleaner = re.compile('<.*?>')
    for review in whole_training_data:
        cleantext = re.sub(cleaner, '', review)
        no_HTML_whole_training_data.append(cleantext)
    return no_HTML_whole_training_data

#Remove stop words and punctuation
def remove_punc_stop_words(no_HTML_whole_training_data):
    clean_training_data = []
    for review in no_HTML_whole_training_data:
        #remove punctuation marks
        review = review.lower()
        for ch in string.punctuation:
            review = review.replace(ch,' ')
        #Remove stop words
        stop_words = set(stopwords.words('english')) 
        word_tokens = word_tokenize(review) 
        filtered_words = [w for w in word_tokens if not w in stop_words] 
        #construct the review again
        sentence =''
        for word in filtered_words:
            sentence+=word
            sentence+=' '
        clean_training_data.append(sentence)
    return clean_training_data

#clean data from HTML tags
no_HTML_whole_training_data = cleanhtml(whole_training_data)
# print(whole_training_data[210])
# print('\n',no_HTML_whole_training_data[210])

clean_training_data = remove_punc_stop_words(no_HTML_whole_training_data)
# print(set(stopwords.words('english')))
# print('\n',no_HTML_whole_training_data[210])
# print('\n',clean_training_data[210])
# print('\n',no_HTML_whole_training_data[310])
# print('\n',clean_training_data[310])
# print('\n',no_HTML_whole_training_data[4510])
# print('\n',clean_training_data[4510])
no_HTML_whole_test_data = cleanhtml(whole_test_data)
clean_test_data = remove_punc_stop_words(no_HTML_whole_test_data)



In [54]:
#Apply feature extraction: countVectorizer which is based on bag of words algorithm
vectorizer = CountVectorizer()
train_vector = vectorizer.fit_transform(clean_training_data)
test_vector = vectorizer.transform(clean_test_data)
#create labels for the training data and test data first 12500 are positive and the remaining 12500 are negative
train_labels = np.asarray([1]*len(positive_training_data) + [0]*len(negative_training_data))
test_labels = np.asarray([1]*len(positive_test_data) + [0]*len(negative_test_data))

#printing some visualization of the sizes of the train and test vectors 
print(train_labels.shape)
#train_vector has 25000 row corresponding to the 25000 review and 74849 feature extracted
print(train_vector.shape)

#print shapes of test vector and label

print(test_labels.shape)
print(test_vector.shape)

#some visualization for th labels arrays
print(train_labels)
print(test_labels)
logreg = linear_model.LogisticRegression()
logreg.fit(train_vector, train_labels)
print("Logistic Regression classification accuracy:\n",logreg.score(test_vector,test_labels)*100,"%")


(25000,)
(25000, 75684)
(25000,)
(25000, 75684)
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]
Logistic Regression classification accuracy:
 86.332 %
